In [ ]:
import time

In [1]:
def grid(rows, cols):
    return [['P' if (c == 0) and (r == rows - 1) else 'C' for c in range(cols)] for r in range(rows)]

def display(grid, rows, cols):
    for r in range(rows):
        for c in range(cols):
            print(grid[r][c] + "  ", end="")
        print("")

In [ ]:
rows = int(input("Grid rows: "))
cols = int(input("Grid columns: "))
initial_state = grid(rows, cols)
display(initial_state, rows, cols)

In [ ]:
select_row = int(input("Select chocolate row: "))
select_col = int(input("Select chocolate column: "))

while select_row >= rows:
    select_row = int(input("Select chocolate row: "))
while select_col >= cols:
    select_col = int(input("Select chocolate column: "))

